## interval_estimation

In [5]:
def interval_estimation(self, X_new=None, alpha=0.05, sigma=None):
    if self.coefficients is None:
        raise ValueError("Model must be fitted before estimating intervals.")

    n, p = self.X_train.shape  # p includes intercept
    df = n - p
    XtX_inv = np.linalg.inv(self.X_train.T @ self.X_train)
    
    # تقدير التباين
    if sigma is None:
        sigma_hat = np.sqrt(np.sum(self.residuals ** 2) / df)
        crit_value = stats.t.ppf(1 - alpha / 2, df)
    else:
        sigma_hat = sigma
        crit_value = stats.norm.ppf(1 - alpha / 2)
    
    # حساب الخطأ المعياري والهامش
    std_errors = np.sqrt(np.diag(XtX_inv)) * sigma_hat
    margins = crit_value * std_errors

    # فترات الثقة للمعاملات
    lower_bounds = self.coefficients - margins
    upper_bounds = self.coefficients + margins
    coef_names = ['Intercept'] + [f'X{i}' for i in range(1, p)]
    coef_df = pd.DataFrame({
        'Coefficient': coef_names,
        'Estimate': self.coefficients,
        'Std Error': std_errors,
        f'{100*(1-alpha):.1f}% CI Lower': lower_bounds,
        f'{100*(1-alpha):.1f}% CI Upper': upper_bounds
    })

    # فترات التنبؤ
    pred_df = None
    if X_new is not None:
        X_new = np.atleast_2d(X_new)
        if X_new.shape[1] == p - 1:
            X_new = np.c_[np.ones((X_new.shape[0], 1)), X_new]
        elif X_new.shape[1] != p:
            raise ValueError(f"Expected {p - 1} features, got {X_new.shape[1]}")
        
        y_preds = X_new @ self.coefficients
        se_preds = np.sqrt(np.sum(X_new @ XtX_inv * X_new, axis=1)) * sigma_hat
        margin_preds = crit_value * se_preds
        lower_preds = y_preds - margin_preds
        upper_preds = y_preds + margin_preds

        pred_df = pd.DataFrame({
            'Prediction': y_preds,
            f'{100*(1-alpha):.1f}% PI Lower': lower_preds,
            f'{100*(1-alpha):.1f}% PI Upper': upper_preds
        })

    return coef_df, pred_df
